In [ ]:
from __future__ import division, absolute_import, print_function
import sys
import os
from pylab import *  # for plotting
from graph_tool.all import *
import random as rd
import itertools #Biblioteca para geração da lista de elementos repetidos
import numpy as np 
from collections import * #módulo counter para contar o número de elmentos
import csv #Save file in csv

## Número de sítios da rede

Logo de começo devemos restringir o valor de $m$ com relação a $m_0$, fazendo

<center> $ m \le m_0 $, (1) </center>

essa restrição evita ligações em loop do novo sítio. Nós temos que o número de sítos final da rede será dada por

<center> $N = m_0 + t$, (2) </center>

para cada instante de tempo $t$, haverá $m_0+t$ sítios. Com número de ligações dadas por

<center> $l = m_0+mt$, (3)</center>

pois cada um dos $t$ sítios terão $m$ ligações. Para sabermos quanto devemos utilizar no tempo para um número $N$ de sítios (quantas interações devemos fazer), podemos isolar o $t$ em (2),obtendo

<font size="3"><center> $N-m_0 = t$.  </center></font>

A probabilidade de ligação de um novo sítio $i$ é dada por

<font size="3"><center> $p_i = \frac{k_i}{\sum_i k_i}$ </center></font>,

com a distribuição dos graus dados por

<font size="3"><center> p(k) = $\frac{2m_0(m_0+1)}{k(k+1)(k+2)}$ </center></font>



## 2. Algoritmo para rede inicial

Tomando a rede inicial com $m_0$ sítios totalmente conectada (sítios igualmente provaveis inicialmente), teremos a matriz de adjacência dada por

\begin{equation}
A = 
\begin{pmatrix}
0 & 1 & 1 & \cdots & 1\\
1 & 0 & 1 & \cdots & 1\\
1 & 1 & 0 & \cdots & 1\\
1 & 1 & 1 & \ddots & 1\\
1 & 1 & 1 & \cdots & 0\\
\end{pmatrix}
_{(m_0,m_0)},
\end{equation}

A rede totalmente conectada, possuí todos elementos fora da coluna principal igual a 1. Para isso, podemos expressar um algoritmo primeiro fixando não-ligação para $i=j$ (sem loops), e logo em seguinda tomando a condição de que $j\le i$, para pegar apenas os elementos do triângulo superior da matriz.

### 2.1 Desenvolvimento da rede

O método para gerar a rede de uma forma eficiente é através da construção de um array $K$. Esse array $K$ guardará as informações acerca do número de ligações que cada sítio $i$ fará, da seguinte forma:

* Cada sítio $i$, representado por um valor número, aparecerá em $K$ num número de vezes igual o seu número de ligações $k$, ou seja, caso os sítios 0, 1 e 2, possuam 1, 2 e 2 ligações, respectivamente, nós teremos o K para essa rede dada por

<center> $K = [0,1,1,2,2]  $</center>,

vemos que o comprimento de K (5 elementos), representa o grau total da rede ($\sum_j k_j$), com o número de vezes que o elemento aparece representando o $k_i$. Ao selecionar aleatoriamente um elemento de $K$, nós temos que os valores que mais aparecem (sítios com mais ligações), tem mais probabilidades de serem selecionados. Satisfazendo a condição $p_i$ imposta sobre essa rede em especifico



In [ ]:
#m0: número de sítios' iniciais;
def barabasi_network(m0,m,N,s):
    seed(s)
    g = Graph(directed=False) #Grafo sem direção
    
    #S = g.new_vertex_property("double")
    #L = g.new_edge_property("double")
    
    g.add_vertex(m0) #Add m0 elements de sítios em g;

    K = [] #Lista que guardará os os valores dos sítios i, k vezes;
    K_supli = [] #Lista Suplementar, que juntará os novos elementos ao K;
    K_filter = [] #Lista que será filtrada para ser usada dentro do loop;
    
    #Criando a rede inicial----------------------------------------------------------
    ver = np.zeros(N) #Guardará os sítios num array;
    
    #Rede inicial totalmente conectada;
    for i in range(m0):
        ver[i] = int(g.vertex(i)) # Definindo todos valores dos sítios i como inteiros
        for j in range(i+1,m0):
            g.add_edge(g.vertex(i),g.vertex(j)) #Construindo uma rede totalmente ligada (inicial)
    
    #Construção da lista K inicial--------------------------------------------------
    for i in range(m0):
        #Gera uma  lista de elementos repetidos
        K_supli = list(itertools.repeat(int(g.vertex(i)), int(g.vertex(i).out_degree()))) 
        
        K += K_supli #Add essa lista de elementos repetidos em K
        K_filter += K_supli #Add essa lista de elementos repetidos em K
        
    #---------------------------------Crescimento da rede-------------------------------
    for i in range(m0,N):
        v = g.add_vertex() #Adiciona um novo sítio
        
        # O for abaixo é responsável por sortear um sítio dentre todos existentes da rede (antes da adição
        # do novo sítio), e realizar uma ligação desse sítio sorteado com o sítio adicionado. Após isso, é
        #feito uma filtragem, deixando apenas os sítios restantes (evitar sítios com duas ligações), essa filtragem
        #é feita m vezes.
        
        for j in range(m):
            Num = randint(0,len(K_filter)-1) #Gera um número aleatório(sítio) dentro de K_filter ;
            u = g.vertex(K_filter[Num]) #seleciona o sítio que foi gerado acima (transforma int em vertex);
            L = g.add_edge(v,g.vertex(K_filter[Num])) #Realiza a ligação do sítio add como o sítio sorteado;
            K += [int(u)] #Adiciona esse sítio sorteado aos valores de K;
            #Filtra K, removendo o sítio já sorteado;
            K_filter = list(filter(lambda val: val !=  int(u), K_filter)) 
        
        #Constroi uma lista com as m ligações do sítio novo (v)
        K_supli = list(itertools.repeat(int(v), int(v.out_degree())))
        K += K_supli #Add a lista acima em K
        #Atualiza K_filter, add os novos sítios (v) e retomando K_filter a forma original (antes da filtragem)
        K_filter = K  
    
    return K

In [ ]:
K_t1 = barabasi_network(5,3,10000,1001) #gerando uma rede de 10.0000 elementos;

In [ ]:
N = 10000 #Número de sítios
L = list(Counter(K_t1).values()) #Número de graus para cada sítio;
print(L)

In [ ]:
#Histograma de L
plt.hist(L,bins=200)
plt.xlim(xmin=0, xmax = 50)
plt.show()

In [ ]:
#histograma Log-Log de L
hist, bins, _ = plt.hist(L, bins=8)
logbins = np.logspace(np.log10(bins[0]),np.log10(bins[-1]),len(bins))
plt.subplot(212)
plt.hist(L, bins=logbins)
plt.xscale('log')
plt.yscale('log')
plt.show()

In [ ]:
#Gerando as 500 implementações (redes) para difentes tamanhos de redes o grau de cada sítio;

m0 = 5 #Número de sítios iniciais
m = 3 #Número de ligação para cada novo sítio

Nimp = 500 #Número de implementações
N1K = 1000 #Número de sítios da rede final: 1.000 Sítios
N10K = 10000 #Número de sítios da rede final: 10.000 Sítios

for i in range(Nimp):
    degree1K = list(Counter(barabasi_network(m0,m,N1K,i)).values())
    degree10K = list(Counter(barabasi_network(m0,m,N10K,i+Nimp)).values())
    np.savetxt('/home/junior/codes/barabasi_model/data/N1K/data.csv', degree1K, fmt="%i", delimiter=',')
    np.savetxt('/home/junior/codes/barabasi_model/data/N10K/data.csv', (degree10K,), delimiter=',')


In [ ]:
m0 = 5 #Número de sítios iniciais
m = 3 #Número de ligação para cada novo sítio
N1K = 1000 #Número de sítios da rede final: 1.000 Sítios
Nimp =  2

degree1K = np.array(Counter(barabasi_network(m0,m,N1K,100000)).values())
np.savetxt('/home/junior/codes/barabasi_model/data/N1K/data.csv', degree1K,fmt='%i', delimiter=',')

#for i in range(Nimp):
#    degree1K,),fmt="%i", delimiter=',')

In [ ]:
Nimp = 500 #Número de implementações
N1kk = 100000 #100.000 Sítios
N1kk = 500000 #500.000 Sítios